In [1]:
import logging

import probtorch
import torch

import combinators
import filtering
import hmm
import utils

logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [2]:
generative = combinators.ParticleTrace(1)

In [3]:
hmm_params = utils.vardict({
    'mu': {
        'loc': torch.arange(5, dtype=torch.float) * 2,
        'scale': torch.ones(5) * 0.25,
    },
    'sigma': {
        'loc': torch.ones(5),
        'scale': torch.ones(5) * 0.25,
    }
})
for k in range(6):
    hmm_params['Pi_%d' % k] = {'concentration': torch.ones(5)}

In [4]:
init_hmm = combinators.Model(f=hmm.init_hmm, hyper=hmm_params)

In [5]:
z0, mu, sigma, pi, pi0 = init_hmm(trace=generative)

In [6]:
hmm_step = combinators.Model(f=hmm.hmm_step)
hmm_run = combinators.Model.sequence(hmm_step, 50, z0, mu, sigma, pi, pi0)

In [7]:
z_last, mu, sigma, pi, pi0 = hmm_run(trace=generative)

In [8]:
data = generative.unwrap(lambda k, rv: 'X_' in k)

In [9]:
num_particles = 100

In [10]:
inference = combinators.ConditionedTrace(100, data=data)

In [11]:
hmm_params = utils.vardict({
    'mu': {
        'loc': torch.arange(5, dtype=torch.float) * 2,
        'scale': torch.ones(5) * 0.25,
    },
    'sigma': {
        'loc': torch.ones(5),
        'scale': torch.ones(5) * 0.25,
    }
})
for k in range(6):
    hmm_params['Pi_%d' % k] = {'concentration': torch.ones(5)}
    hmm_params['Pi_%d' % k]['concentration'][1] = 1.25
    hmm_params['Pi_%d' % k]['concentration'][2] = 1.5
    hmm_params['Pi_%d' % k]['concentration'][3] = 1.25

In [12]:
init_hmm = combinators.Model(f=hmm.init_hmm, trainable=hmm_params)

In [13]:
z0, mu, sigma, pi, pi0 = init_hmm(trace=inference)

In [14]:
hmm_step = hmm.forward_backward_filter_hmm(mu, sigma, pi, pi0)
hmm_run = combinators.Model.sequence(hmm_step, 50, z0, mu, sigma, pi, pi0)

In [15]:
z_last, mu, sigma, pi, pi0 = hmm_run(trace=inference)

In [16]:
hmm_step.backward_pass(T=50)

In [17]:
hmm_step.smoothed_posterior(T=50)[0][3]

tensor([[ -3.5195,  -1.7482,  -0.2542,  -3.9374,  -6.6779],
        [ -2.8917,  -1.7672,  -0.2595,  -6.2191,  -8.3022],
        [ -3.6359,  -2.3655,  -0.2165,  -2.6877,  -5.0609],
        [ -0.7560,  -1.4895,  -1.1875, -20.8954, -19.9060],
        [ -5.0111,  -4.4336,  -1.1235,  -0.5212,  -2.7720],
        [ -0.1719,  -2.6226,  -2.4612, -30.5153, -27.5216],
        [ -9.7582, -11.9193,  -5.1678,  -2.5930,  -0.0840],
        [ -0.2140,  -1.8743,  -3.2390, -38.2945, -32.7587],
        [ -4.9586,  -3.6866,  -1.2418,  -0.5954,  -2.0580],
        [ -1.3270,  -0.5270,  -1.9355, -20.1886, -19.4651],
        [ -1.1653,  -0.6415,  -1.8222, -18.0236, -19.4429],
        [ -0.0062,  -5.2648,  -6.8780, -55.1770, -42.2433],
        [ -3.0761,  -2.4353,  -0.1744,  -3.7049,  -6.3542],
        [ -0.7444,  -1.4709,  -1.2198, -22.3141, -20.5230],
        [ -2.6257,  -2.4124,  -0.1796,  -6.0962,  -8.6204],
        [ -8.3783,  -9.8306,  -2.9469,  -0.1462,  -2.4861],
        [-17.3883, -25.3440, -11.9579, -

In [18]:
init_hmm_params = utils.vardict({
    'mu': {
        'loc': torch.rand(5) * 10,
        'scale': torch.ones(5),
    },
    'sigma': {
        'loc': torch.ones(5),
        'scale': torch.ones(5),
    }
})
for k in range(6):
    init_hmm_params['Pi_%d' % k] = {'concentration': torch.ones(5)}

In [19]:
init_hmm = combinators.Model(f=hmm.init_hmm, trainable=init_hmm_params)

In [20]:
def hmm_step_builder(z0, mu, sigma, pi, pi0):
    return hmm.forward_backward_filter_hmm(mu, sigma, pi, pi0)

In [21]:
inference, variational_params = filtering.variational_forward_backward(init_hmm, hmm_step_builder, 500, 50, data, use_cuda=False, lr=1e-2)

09/16/2018 18:59:25 Variational forward-backward ELBO=-7.84629785e+03 at epoch 1
09/16/2018 18:59:25 Variational forward-backward ELBO=-6.81461182e+03 at epoch 2
09/16/2018 18:59:26 Variational forward-backward ELBO=-7.69350830e+03 at epoch 3
09/16/2018 18:59:26 Variational forward-backward ELBO=-7.56364600e+03 at epoch 4
09/16/2018 18:59:26 Variational forward-backward ELBO=-7.22110010e+03 at epoch 5
09/16/2018 18:59:27 Variational forward-backward ELBO=-7.25064453e+03 at epoch 6
09/16/2018 18:59:27 Variational forward-backward ELBO=-8.16878369e+03 at epoch 7
09/16/2018 18:59:27 Variational forward-backward ELBO=-7.03737891e+03 at epoch 8
09/16/2018 18:59:28 Variational forward-backward ELBO=-7.06978613e+03 at epoch 9
09/16/2018 18:59:28 Variational forward-backward ELBO=-8.06237158e+03 at epoch 10
09/16/2018 18:59:29 Variational forward-backward ELBO=-7.41913477e+03 at epoch 11
09/16/2018 18:59:29 Variational forward-backward ELBO=-8.14053564e+03 at epoch 12
09/16/2018 18:59:29 Varia

In [22]:
variational_params

"{'Pi_0__concentration': 'Parameter containing:\ntensor([0.6596, 0.5699, 0.9785, 1.7031, 1.1632], requires_grad=True)', 'Pi_1__concentration': 'Parameter containing:\ntensor([0.3343, 1.5660, 0.8396, 1.1276, 1.8431], requires_grad=True)', 'Pi_2__concentration': 'Parameter containing:\ntensor([0.4029, 1.0605, 0.3682, 0.5144, 2.3627], requires_grad=True)', 'Pi_3__concentration': 'Parameter containing:\ntensor([1.4761, 1.5671, 0.5190, 1.1646, 1.1175], requires_grad=True)', 'Pi_4__concentration': 'Parameter containing:\ntensor([1.4190, 1.5200, 0.9862, 0.8437, 1.0490], requires_grad=True)', 'Pi_5__concentration': 'Parameter containing:\ntensor([1.2256, 2.1633, 0.6120, 0.5859, 1.4699], requires_grad=True)', 'mu__loc': 'Parameter containing:\ntensor([5.1654, 4.2896, 5.6224, 5.4701, 0.5132], requires_grad=True)', 'mu__scale': 'Parameter containing:\ntensor([-0.2414, -0.2461,  0.2215,  0.4239, -0.0424], requires_grad=True)', 'sigma__loc': 'Parameter containing:\ntensor([1.5474, 1.5385, 1.8992, 1